In [1]:
import numpy as np
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp

In [ ]:
class SGD:
    """Stochastic Gradient Descent optimizer with parameter shift method."""

    def __init__(self, ansatz, hamiltonian, estimator, learning_rate=0.01):
        self.ansatz = ansatz
        self.hamiltonian = hamiltonian
        self.estimator = estimator
        self.learning_rate = learning_rate

    def minimize(self, initial_params, num_iterations=100):
        params = initial_params
        for i in range(num_iterations):
            params = self.step(params)
            if (i + 1) % 10 == 0:
                cost = self.cost_function(params)
                print(f"Iteration {i+1}: Cost = {cost}")
        return params

    def minimize_adaptive(self, initial_params, window_size=10, ef=1e-4, et=5e-4, num_iterations=100):
        params = initial_params
        cost_window = []
        param_window = []
        for i in range(num_iterations):
            params = self.step(params)
            cost = self.cost_function(params)
            if (i + 1) % 10 == 0:
                print(f"Iteration {i+1}: Cost = {cost}")

            cost_window.append(cost)
            param_window.append(params)
            if i >= window_size:
                energy_diff = abs(np.mean(cost_window[-window_size:]) - 
                                 np.mean(cost_window[-window_size-1:-1]))
                param_diff = np.linalg.norm(
                    np.mean(param_window[-window_size:], axis=0) - 
                    np.mean(param_window[-window_size-1:-1], axis=0)
                )
                
                if energy_diff < ef or param_diff < et:
                    break
        return params

    def step(self, params):
        grad = self.parameter_shift_grad_v2(params)
        return params - self.learning_rate * grad

    def parameter_shift_grad(self, params):
        grad = np.zeros(len(params))
        for i in range(len(params)):
            shifted = params.copy()
            shifted[i] += np.pi/2
            plus = self.cost_function(shifted)
            
            shifted = params.copy()
            shifted[i] -= np.pi/2
            minus = self.cost_function(shifted)
            
            grad[i] = 0.5 * (plus - minus)
        return grad
    
    def parameter_shift_grad_v2(self, params):
        n = len(params)
        pubs = []
        # build batch of plus/minus shifted parameters
        for i in range(n):
            plus = params.copy()
            plus[i] += np.pi/2
            pubs.append((self.ansatz, [self.hamiltonian], [plus]))
            minus = params.copy()
            minus[i] -= np.pi/2
            pubs.append((self.ansatz, [self.hamiltonian], [minus]))
        # run one batched call
        results = self.estimator.run(pubs=pubs).result()
        # extract energies and form gradient
        grad = np.zeros(n)
        for i in range(n):
            e_plus = results[2*i].data.evs[0]
            e_minus = results[2*i+1].data.evs[0]
            grad[i] = 0.5 * (e_plus - e_minus)
        return grad
    
    def cost_function(self, params):
        pub = (self.ansatz, [self.hamiltonian], [params])
        result = self.estimator.run(pubs=[pub]).result()[0]
        return result.data.evs[0]

In [3]:
estimator = Estimator()
estimator.options.run_options = {"shots": 10_000}

In [4]:
def get_ansatz():
    params = ParameterVector("θ", 2)
    ansatz = QuantumCircuit(2)
    ansatz.h(0)
    ansatz.cx(0, 1)
    ansatz.rx(params[0], 0)
    ansatz.ry(params[1], 1)
    return ansatz

In [5]:
ansatz = get_ansatz()
num_params = ansatz.num_parameters

In [6]:
hamiltonian = SparsePauliOp.from_list([("ZZ", 1.0)])

In [24]:
optimizer = SGD(ansatz, hamiltonian, estimator, learning_rate=0.4)

In [19]:
params = 2 * np.pi * np.random.random([num_params])

In [25]:
print("Initial parameters:", params)
print("Initial cost:", optimizer.cost_function(params))
print("Initial gradient:", optimizer.parameter_shift_grad(params))

Initial parameters: [1.14368826 4.17852533]
Initial cost: -0.21079167212100283
Initial gradient: [0.4631507  0.35659778]


In [281]:
# for i in range(100):
#     params = optimizer.step(params)
#     if (i + 1) % 10 == 0:
#         print(f"Iteration {i+1}:")
#         # print("Parameters:", params)
#         print("Cost:", optimizer.cost_function(params))
#         # print("Gradient:", optimizer.parameter_shift_grad(params))

In [15]:
optimizer.minimize_adaptive(params)

Iteration 10: Cost = -0.98607766359862
Iteration 20: Cost = -0.9999994843408506


array([2.17149700e-04, 3.14162369e+00])

In [26]:
optimizer.minimize(params)

Iteration 10: Cost = -0.9996621944047803
Iteration 20: Cost = -0.9999999876421086
Iteration 30: Cost = -0.999999999999548
Iteration 40: Cost = -1.0
Iteration 50: Cost = -1.0
Iteration 60: Cost = -1.0
Iteration 70: Cost = -1.0
Iteration 80: Cost = -1.0
Iteration 90: Cost = -1.0
Iteration 100: Cost = -1.0


array([7.55679330e-17, 3.14159265e+00])